# Confidence Score Modeling (信頼度算出モデルの構築)
本ノートブックでは、算出されたコサイン類似度の差分（マージン）を「炎上リスクの確率（0.0〜1.0）」へと最適にマッピングするためのキャリブレーション・モデル（ロジスティック回帰）を構築します。これにより、単なるスコア比較を超えた、統計的根拠に基づくリスク判定を実現します。

## 1. Environment Setup & Data Loading
解析に必要なライブラリのインポート、事前学習済みBERTモデルの初期化、および前工程（03_vector_caching）で生成した参照用ベクトルのロードを行います。

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
from transformers import BertTokenizer, BertModel
import random
import os
from sklearn.linear_model import LogisticRegression
import pickle
from sklearn.model_selection import train_test_split

# BERTモデルおよびトークナイザーの初期化 (bert-base-uncased)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', model_max_length=512)
model = BertModel.from_pretrained('bert-base-uncased')

# 計算リソースの最適化（CUDA環境の優先活用）
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

## 2. Evaluation Logic Definition
分類器の推論精度を定量的評価し、信頼度算出モデルの学習用データとして、正解・不正解時の類似度分布を抽出するための関数を定義します。

In [ ]:
def check_classification(answer_label, prediction_labels, dif_similarity_scores, most_similarity_scores):
  """
  推論結果の正誤判定を行い、特徴量（類似度マージンおよび最大類似度）を各クラスに分配する。
  """
  true_num, false_num = 0, 0
  true_dif_sims, false_dif_sims = [], []
  true_most_sims, false_most_sims = [], []

  for k in range(len(prediction_labels)):
    # 予測がラベルと一致（正解）した場合の統計値抽出
    if answer_label == prediction_labels[k]:
      true_num += 1
      true_dif_sims.append(dif_similarity_scores[k])
      true_most_sims.append(most_similarity_scores[k])
    # 予測がラベルと不一致（不正解）した場合の統計値抽出
    else:
      false_num += 1
      false_dif_sims.append(dif_similarity_scores[k])
      false_most_sims.append(most_similarity_scores[k])

  return true_num, false_num, true_dif_sims, false_dif_sims, true_most_sims, false_most_sims

## 3. Similarity-based Classification Engine
コサイン類似度に基づき、未知のテキスト（テストデータ）が「SAFE」または「OUT」のどちらの参照クラスタに属するかを判定する推論パイプラインを実装します。

In [ ]:
def get_similarity(test_vectors, test_label, train_safe_vectors, train_out_vectors):
  """
  ベクトル間のコサイン類似度をバッチ処理で計算し、k-NN的なアプローチでクラス分類を行う。
  """
  # 各ベクトルを計算用テンソルへ変換
  test_v_all = torch.tensor(np.array(test_vectors)).to(device)
  safe_v_all = torch.tensor(np.array(train_safe_vectors)).to(device)
  out_v_all = torch.tensor(np.array(train_out_vectors)).to(device)

  batch_size = 64
  source_files, dif_similarity_scores, most_similarity_scores, other_similarity_scores = [], [], [], []
  error = 0

  for i in tqdm(range(0, len(test_v_all), batch_size)):
    batch_test = test_v_all[i : i + batch_size]

    # 行列演算によるコサイン類似度の一括計算
    sim_matrix_safe = torch.nn.functional.cosine_similarity(batch_test.unsqueeze(1), safe_v_all.unsqueeze(0), dim=2)
    sim_matrix_out = torch.nn.functional.cosine_similarity(batch_test.unsqueeze(1), out_v_all.unsqueeze(0), dim=2)

    # 各バッチにおける最大類似度の取得
    max_sim_safe_batch, _ = torch.max(sim_matrix_safe, dim=1)
    max_sim_out_batch, _ = torch.max(sim_matrix_out, dim=1)

    for j in range(len(batch_test)):
      ms_safe = max_sim_safe_batch[j].item()
      ms_out = max_sim_out_batch[j].item()

      # スコア比較に基づく決定論的分類
      if ms_safe > ms_out:
          source_files.append("safe")
          most_similarity_scores.append(ms_safe)
          other_similarity_scores.append(ms_out)
      elif ms_safe < ms_out:
          source_files.append("out")
          most_similarity_scores.append(ms_out)
          other_similarity_scores.append(ms_safe)
      else:
          source_files.append("error")
          most_similarity_scores.append(0); other_similarity_scores.append(0)
          error += 1

      # 信頼度算出の鍵となる「類似度マージン」を算出
      dif_similarity_scores.append(most_similarity_scores[-1] - other_similarity_scores[-1])

  return check_classification(test_label, source_files, dif_similarity_scores, most_similarity_scores)

## 4. Execution & Dataset Balancing
クラス間のサンプル数不均衡（Class Imbalance）を解消するため、アンダーサンプリングを実施した上で推論を実行し、キャリブレーション用の学習データを生成します。

In [ ]:
# 1. 保存済みキャッシュデータのロード
with open("bert_vectors_data.pkl", 'rb') as f:
    cache_data = pickle.load(f)

safe_vec_all = cache_data['safe_vec_all']
out_vec_all = cache_data['out_vec_all']

# 2. クラス不均衡の是正（最小サンプル数へのランダム・アンダーサンプリング）
random.seed(1)
min_size = min(len(safe_vec_all), len(out_vec_all))

safe_indices = random.sample(range(len(safe_vec_all)), min_size)
safe_vec = [safe_vec_all[i] for i in safe_indices]

out_indices = random.sample(range(len(out_vec_all)), min_size)
out_vec = [out_vec_all[i] for i in out_indices]

# ホールドアウト法による学習・テストデータの分割 (10%を検証用に使用)
test_size = 0.1
random_state = 1

safe_train_vec, safe_test_vec = train_test_split(
    safe_vec, test_size=test_size, random_state=random_state
)

out_train_vec, out_test_vec = train_test_split(
    out_vec, test_size=test_size, random_state=random_state
)

# 3. 分類実行と特徴量抽出の開始
safe_true, safe_false, safe_true_dif_sims, safe_false_dif_sims, safe_true_most_sims, safe_false_most_sims = get_similarity(safe_test_vec, "safe", safe_train_vec, out_train_vec)
out_true, out_false, out_true_dif_sims, out_false_dif_sims, out_true_most_sims, out_false_most_sims = get_similarity(out_test_vec, "out", safe_train_vec, out_train_vec)

## 5. Feature Engineering for Calibration Model
信頼度算出モデル（ロジスティック回帰）の入力となる説明変数（特徴量）と目的変数を整形します。

In [ ]:
# SAFE予測時における確率マッピング用データセットの作成
X_safe_pred = np.array([
    [d, m] for d, m in zip(safe_true_dif_sims, safe_true_most_sims)
] + [
    [d, m] for d, m in zip(out_false_dif_sims, out_false_most_sims)
])
y_safe_pred = np.array([1] * len(safe_true_dif_sims) + [0] * len(out_false_dif_sims))

# OUT予測時における確率マッピング用データセットの作成
X_out_pred = np.array([
    [d, m] for d, m in zip(out_true_dif_sims, out_true_most_sims)
] + [
    [d, m] for d, m in zip(safe_false_dif_sims, safe_false_most_sims)
])
y_out_pred = np.array([1] * len(out_true_dif_sims) + [0] * len(safe_false_dif_sims))

## 6. Logistic Regression for Confidence Calibration
抽出した類似度の統計量から、最終的な「炎上リスク確率」を導出するための数式モデル（重み・切片）を学習・保存します。

In [ ]:
# ロジスティック回帰によるキャリブレーション・モデルの学習
model_safe = LogisticRegression().fit(X_safe_pred, y_safe_pred)
model_out = LogisticRegression().fit(X_out_pred, y_out_pred)

# 推論アプリで使用するための学習済みパラメータの構造化
params = {
    'safe_params': {
        'intercept': model_safe.intercept_[0],
        'coef_dif': model_safe.coef_[0][0],
        'coef_most': model_safe.coef_[0][1]
    },
    'out_params': {
        'intercept': model_out.intercept_[0],
        'coef_dif': model_out.coef_[0][0],
        'coef_most': model_out.coef_[0][1]
    }
}

# 既存のラベルデータにキャリブレーション・パラメータを統合して永続化
with open("texts_label_data.pkl", 'rb') as f:
    existing_data = pickle.load(f)
existing_data.update(params)
with open("texts_label_data.pkl", 'wb') as f:
    pickle.dump(existing_data, f)